In [1]:
import random
from collections import Counter
from tqdm import tqdm

import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import esm

In [2]:
import scipy
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC, SVR
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, SGDRegressor

In [3]:
import random
from collections import Counter
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import esm
import scipy
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC, SVR
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, SGDRegressor
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.preprocessing import LabelBinarizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import torch.nn as nn
from itertools import chain

In [4]:
df = pd.read_csv('/home/user/Projects/Paper_Noelia/my_code/ESM_Model_test/input_prot_fam/pdb_data_no_dups.csv').merge(pd.read_csv('/home/user/Projects/Paper_Noelia/my_code/ESM_Model_test/input_prot_fam/pdb_data_seq.csv'), how='inner', on='structureId').drop_duplicates(["sequence"]) # ,"classification"
# Drop rows with missing labels
df = df[[type(c) == type('') for c in df.classification.values]]
df = df[[type(c) == type('') for c in df.sequence.values]]
# select proteins
df = df[df.macromoleculeType_x == 'Protein']
df.reset_index()
print(df.shape)
df.head()

(87761, 18)


,structureId,classification,experimentalTechnique,macromoleculeType_x,residueCount_x,resolution,structureMolecularWeight,crystallizationMethod,crystallizationTempK,densityMatthews,densityPercentSol,pdbxDetails,phValue,publicationYear,chainId,sequence,residueCount_y,macromoleculeType_y
4,101M,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,154,2.07,18112.80,NaN,NaN,3.09,60.20,"3.0 M AMMONIUM SULFATE, 20 MM TRIS, 1MM EDTA, ...",9.0,1999.0,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,Protein
7,102L,HYDROLASE(O-GLYCOSYL),X-RAY DIFFRACTION,Protein,165,1.74,18926.61,NaN,NaN,2.75,55.28,NaN,NaN,1993.0,A,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSE...,165,Protein
8,102M,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,154,1.84,18010.64,NaN,NaN,3.09,60.20,"3.0 M AMMONIUM SULFATE, 20 MM TRIS, 1MM EDTA, ...",9.0,1999.0,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,Protein
11,103L,HYDROLASE(O-GLYCOSYL),X-RAY DIFFRACTION,Protein,167,1.90,19092.72,NaN,NaN,2.70,54.46,NaN,NaN,1993.0,A,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNSLDAAK...,167,Protein
15,104L,HYDROLASE(O-GLYCOSYL),X-RAY DIFFRACTION,Protein,332,2.80,37541.04,NaN,NaN,3.04,59.49,NaN,NaN,1993.0,A,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSAA...,332,Protein


In [5]:
print(df.residueCount_x.quantile(0.9))
df.residueCount_x.describe()

1732.0


count     87761.000000
mean        922.913937
std        3173.118920
min           3.000000
25%         234.000000
50%         451.000000
75%         880.000000
max      157478.000000
Name: residueCount_x, dtype: float64

In [6]:
df = df.loc[df.residueCount_x<1200]
print(df.shape[0])
df.residueCount_x.describe()

73140


count    73140.000000
mean       433.599918
std        289.671609
min          3.000000
25%        198.000000
50%        373.000000
75%        618.000000
max       1198.000000
Name: residueCount_x, dtype: float64

In [7]:
# count numbers of instances per class
cnt = Counter(df.classification)
# select only K most common classes! - was 10 by default
top_classes = 10
# sort classes
sorted_classes = cnt.most_common()[:top_classes]
classes = [c[0] for c in sorted_classes]
counts = [c[1] for c in sorted_classes]
print("at least " + str(counts[-1]) + " instances per class")

at least 1416 instances per class


In [8]:
print(str(df.shape[0]) + " instances before")
df = df[[c in classes for c in df.classification]]
print(str(df.shape[0]) + " instances after")

seqs = list(df.sequence)
lengths = [len(s) for s in seqs]

73140 instances before
37280 instances after


In [10]:
max_length = 300
for n,i in enumerate(seqs):
    seqs[n] = seqs[n][:max_length]

In [11]:
ys = list(df.classification)
len(ys)

37280

In [13]:
FASTA_PATH = "/home/user/Projects/Paper_Noelia/my_code/ESM_Model_test/input_prot_fam/df_prot_fam.fasta" 
EMB_PATH = "/home/user/Projects/Paper_Noelia/my_code/ESM_Model_test/input_prot_fam/Results_bulk_emb_esm" 
EMB_LAYER = 33

In [14]:
Xs = []
for header, _seq in esm.data.read_fasta(FASTA_PATH):
    fn = f'{EMB_PATH}/{header[1:]}.pt'
    embs = torch.load(fn)
    Xs.append(embs['mean_representations'][EMB_LAYER])
Xs = torch.stack(Xs, dim=0).numpy()
print(Xs.shape)

(37280, 1280)


In [15]:
train_size = 0.8
Xs_train, Xs_test, ys_train, ys_test = train_test_split(Xs, ys, train_size=train_size, random_state=42)

In [16]:
Xs_train.shape, Xs_test.shape, len(ys_train), len(ys_test)

((29824, 1280), (7456, 1280), 29824, 7456)

In [17]:
pca = PCA(60)
Xs_train_pca = pca.fit_transform(Xs_train)

In [18]:
Xs_train[0].size

1280

In [19]:
Xs_train

array([[ 0.1438748 ,  0.03533748, -0.02898256, ..., -0.09603556,
        -0.05697378,  0.19259219],
       [ 0.17272273,  0.16387296, -0.04663553, ...,  0.11809193,
        -0.08998685,  0.14496161],
       [ 0.0222279 ,  0.2175613 , -0.12115411, ...,  0.12973131,
        -0.08974393, -0.14509784],
       ...,
       [ 0.15982424,  0.11773956, -0.04919418, ..., -0.09676334,
         0.137497  ,  0.15694834],
       [ 0.06932002,  0.3028572 , -0.05969365, ..., -0.17375427,
        -0.18557784,  0.05983735],
       [ 0.01167518,  0.17601773, -0.04654519, ..., -0.00160343,
        -0.05102926,  0.0152693 ]], dtype=float32)

In [20]:
Xs_test_pca = pca.transform(Xs_test)

In [21]:
knn = KNeighborsClassifier(n_neighbors = 5) #setting up the KNN model to use 5NN
knn.fit(Xs_train_pca, ys_train) #fitting the KNN

KNeighborsClassifier()

In [22]:
#Checking performance on the training set
print('Accuracy of K-NN classifier on training set: {:.2f}'
     .format(knn.score(Xs_train_pca, ys_train)))

Accuracy of K-NN classifier on training set: 0.82


In [23]:
#Checking performance on the test set
print('Accuracy of K-NN classifier on test set: {:.2f}'
     .format(knn.score(Xs_test_pca, ys_test)))

Accuracy of K-NN classifier on test set: 0.74


In [24]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(Xs_train_pca, ys_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [25]:
print('Accuracy of SCV classifier on training set: {:.2f}'
     .format(clf.score(Xs_train_pca, ys_train)))

Accuracy of SCV classifier on training set: 0.82


In [26]:
print('Accuracy of SCV classifier on test set: {:.2f}'
     .format(clf.score(Xs_test_pca, ys_test)))

Accuracy of SCV classifier on test set: 0.77


In [27]:
rfc = RandomForestClassifier(max_depth=2, random_state=0)
rfc.fit(Xs_train_pca, ys_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [29]:
print('Accuracy of Random Forest classifier on training set: {:.2f}'
     .format(rfc.score(Xs_train_pca, ys_train)))

Accuracy of Random Forest classifier on training set: 0.36


In [28]:
print('Accuracy of Random Forest classifier on test set: {:.2f}'
     .format(rfc.score(Xs_test_pca, ys_test)))

Accuracy of Random Forest classifier on test set: 0.37
